## 추가 변수 생성 및 결측치 처리

In [128]:
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")

In [129]:
## total_busdata 
total_busdata = pd.read_csv(
    filepath_or_buffer="../data/total_busdata.csv"
    )

### 첫차, 막차시간 변수 결측값 처리

In [130]:
# 시간 변환 함수 정의

# to minutes
def convert_to_minutes(time):
    if pd.isna(time):  # NaN 값인지 확인
        return time  # NaN 값을 그대로 반환
    hours = int(time // 100)
    minutes = int(time % 100)
    return hours * 60 + minutes

# to hours
def convert_to_hours_minutes(minutes):
    hours = int(minutes // 60)
    remaining_minutes = int(minutes % 60)
    return hours * 100 + remaining_minutes


In [131]:
# startvehicletime 과 endvehicletime 변수의 시간 변환 값을 각각 start, end 변수에 넣음

total_busdata['start'] = total_busdata['startvehicletime'].apply(convert_to_minutes)
total_busdata['end'] = total_busdata['endvehicletime'].apply(convert_to_minutes)

In [132]:
# start, end 열의 평균 
mean_start = total_busdata['start'].mean()
mean_end = total_busdata['end'].mean()

# start, end 열의 결측값에 평균 대입
total_busdata['start'] = total_busdata['start'].fillna(mean_start)
total_busdata['end'] = total_busdata['end'].fillna(mean_end)

# start, end 열의 평균을 원래 시간 형태로 변환
mean_start = convert_to_hours_minutes(mean_start)
mean_end = convert_to_hours_minutes(mean_end)

In [133]:
# startvehicletime과 endvehicletime의 결측값에 대입
total_busdata['startvehicletime'] = total_busdata['startvehicletime'].fillna(mean_start)
total_busdata['endvehicletime'] = total_busdata['endvehicletime'].fillna(mean_end)

### 배차간격 결측값 처리

In [134]:
# 배차간격 결측값 : 시,군 전체 평균을 결측값에 넣음
intervaltime_avg = total_busdata['intervaltime'].mean()
total_busdata['intervaltime'] = total_busdata['intervaltime'].fillna(intervaltime_avg)

intervalsattime_avg = total_busdata['intervalsattime'].mean()
total_busdata['intervalsattime'] = total_busdata['intervalsattime'].fillna(intervalsattime_avg)

intervalsuntime_avg = total_busdata['intervalsuntime'].mean()
total_busdata['intervalsuntime'] = total_busdata['intervalsuntime'].fillna(intervalsuntime_avg)

total_busdata[['intervaltime','intervalsattime','intervalsuntime']]= total_busdata[['intervaltime','intervalsattime','intervalsuntime']].round(1)

In [135]:
total_busdata

,city_name,routeno,routetp,startnodenm,endnodenm,startvehicletime,endvehicletime,intervaltime,intervalsattime,intervalsuntime,usage,노선정류장수,노선거리(km),굴곡도,start,end
0,창원시,1,마을버스,신전,창원역,540.0,2210.0,40.0,40.0,40.0,33931.0,105.0,55.9,2.0,340.000000,1330.000000
1,창원시,2,마을버스,유등,창원역,725.0,2150.0,170.0,170.0,170.0,2558.0,83.0,53.5,1.7,445.000000,1310.000000
2,창원시,3,마을버스,상리,진영시외주차장,750.0,1655.0,220.0,220.0,220.0,299.0,74.0,43.8,2.6,470.000000,1015.000000
3,창원시,7,마을버스,창원역,자여입구,620.0,2310.0,11.0,15.0,15.0,63612.0,35.0,21.1,1.0,380.000000,1390.000000
4,창원시,8,마을버스,창원역,삼성창원병원종점(8),600.0,2240.0,17.0,17.0,17.0,68306.0,39.0,8.8,1.8,360.000000,1360.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508,합천군,660-1,농어촌(일반)버스,합천터미널,합천터미널,928.0,1614.0,128.1,114.6,118.0,NaN,NaN,NaN,NaN,568.676575,974.532234
2509,합천군,670-2,농어촌(일반)버스,합천터미널,합천터미널,928.0,1614.0,128.1,114.6,118.0,NaN,NaN,NaN,NaN,568.676575,974.532234
2510,합천군,810-3,농어촌(일반)버스,부림면정류장,합천터미널,928.0,1614.0,128.1,114.6,118.0,45.0,29.0,25.3,1.3,568.676575,974.532234
2511,합천군,110-13,농어촌(일반)버스,부림면정류장,합천터미널,928.0,1614.0,128.1,114.6,118.0,NaN,NaN,NaN,NaN,568.676575,974.532234


In [136]:
total_busdata.to_csv("../data/여진_total_busdata_na1.csv", index=False)

### 버스대수 변수 생성

In [67]:
# 경남지역 버스의 평균 운행 속도 = (평일*6+토요일+일요일)/7
round((23.6*5+23.8+24.8)/7,1)

23.8

In [ ]:
# 한 노선을 도는데 걸리는 시간
# total_busdata['onetime'] = total_busdata['노선거리']/23.8

In [24]:
# 총 운행 시간 계산 (분 단위)
#total_busdata['total_operating_time'] = total_busdata['end_minutes'] - total_busdata['start_minutes']

# 필요한 버스 대수
#total_busdata['num_bus'] = total_busdata['total_operating_time']/total_busdata['onetime'] 

#### 결측치 종류

1. Missing completely at random (MCAR) - 완전 무작위 결측
결측값의 발생이 다른 변수와 상관이 없는 경우
ex) 전산오류, 통신문제 등으로 데이터 누락

2. Missing at random (MAR) - 무작위 결측
결측값의 발생이 특정 변수와 관련이 있으나 얻고자 하는 결과와는 상관이 없는 경우

3. Not missing at random (NMAR) - 비무작위 결측
결측값 발생이 다른 변수와 상관이 있는 경우



#### 결측치 대체 방법

1. 아무것도 하지 않기

2. 데이터 제거 

3. 중앙값, 평균값, 최빈값, 0, 상수값으로 대체

4. K-NN(K-Nearest Neighbor) 대체 
각 데이터 포인트의 가장 가까운 k개의 이웃을 찾아서 결측값을 예측하는 방식
유클리드 거리를 사용하여 각 노선의 길이, 정류장 수, 첫차 시간, 막차 시간 등 모든 특징을 포함한 거리 계산을 통해 결측값을 대체
-> 결측값이 있는 노선과 가장 유사한 3개의 노선을 찾아 그들의 값을 기반으로 결측값을 예측 

<1>
#from impyute.imputation.cs import fast_knn

#np_imputed = fast_knn(df.values, k=5)
#df_imputed = pd.DataFrame(np_imputed)

<2>

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

#데이터 정규화
scaler = MinMaxScaler()
total_busdata_scaled = scaler.fit_transform(total_busdata)

#KNNImputer 생성 (k=3 사용)
imputer = KNNImputer(n_neighbors=3)

#결측값 대체
total_busdata_filled = imputer.fit_transform(total_busdata_scaled)

#정규화된 데이터를 원래 스케일로 되돌림
total_busdata_filled = scaler.inverse_transform(total_busdata_filled)

#대체된 값을 원래 데이터프레임에 반영
total_busdata['usage'] = total_busdata_filled[:, 0]


5. MICE(Multivariate Imputation by Chained Equation)

#from impyute.imputation.cs import mice

#np_imputed=mice(df.values)
#df_imputed = pd.DataFrame(np_immputed)

6. 딥러닝을 이용한 Imputation (범주형이나 숫자가 아닌 자료형에 효과적)
#import datawig

#imputer = datawig.SimpleImputer(
#input_columns = ['X_1', 'X_2', 'X_3'], # impute에 사용할 col 지정
#output_column = 'X_5') # 컬럼 X_5의 결측치를 채운다

#imputer.fit(train_df=df_null, num_epochs=50)

#df_null_only = df_null[df_null['X_5'].isnull()]

#np_imputed = imputer.predict(df_null_only)
#df_imputed = pd.DataFrame(np_imputed)

